In [1]:
import pandas as pd
import numpy as np

In [5]:
# load dataset
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

In [8]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [9]:
# train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# label encoding - target
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [12]:
import torch

In [16]:
# converting numpy array to torch tensors

X_train_tensor = torch.tensor(X_train)
print("X Train Tensor Shape", X_train_tensor.shape)
X_test_tensor = torch.tensor(X_test)
print("X Test Tensor Shape", X_test_tensor.shape)
y_train_tensor = torch.tensor(y_train)
print("Y Train Tensor Shape", y_train_tensor.shape)
y_test_tensor = torch.tensor(y_test)
print("Y Test Tensor Shape", y_test_tensor.shape)

X Train Tensor Shape torch.Size([455, 30])
X Test Tensor Shape torch.Size([114, 30])
Y Train Tensor Shape torch.Size([455])
Y Test Tensor Shape torch.Size([114])


In [50]:
class SimpleNN():

    def __init__(self, X, learning_rate=0.01, num_epochs=50):
        self.input_size = X.shape[1]
        self.weights = torch.randn(self.input_size, 1, dtype=torch.float32, requires_grad=True) # 30 x 1 : random weights
        self.bias = torch.zeros(1, dtype=torch.float32, requires_grad=True)
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
    
    def forward_pass(self, X):
        z = torch.matmul(X.float(), self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred, y):
        # binary cross entropy loss
        return torch.nn.BCELoss()(y_pred.squeeze(), y.float())
    
    def backward_pass(self, y_pred, y_true):
        # compute loss
        loss = self.loss_function(y_pred, y_true)
        loss.backward() # compute gradients
        
        # update weights
        with torch.no_grad(): # no gradient tracking
            self.weights -= self.learning_rate * self.weights.grad
            self.bias -= self.learning_rate * self.bias.grad

            # zero gradients
            self.weights.grad.zero_()
            self.bias.grad.zero_()
        
        return loss

    def fit(self, X, y):
        # Training over num_epochs
        for i in range(self.num_epochs):
            y_pred = self.forward_pass(X)
            loss = self.backward_pass(y_pred, y)
            print(f"Epoch {i+1}/{self.num_epochs} - Loss: {loss.item():.4f}")

In [51]:
model = SimpleNN(X_train_tensor)

model.fit(X_train_tensor, y_train_tensor)

Epoch 1/50 - Loss: 1.6776
Epoch 2/50 - Loss: 1.6690
Epoch 3/50 - Loss: 1.6606
Epoch 4/50 - Loss: 1.6522
Epoch 5/50 - Loss: 1.6439
Epoch 6/50 - Loss: 1.6357
Epoch 7/50 - Loss: 1.6276
Epoch 8/50 - Loss: 1.6195
Epoch 9/50 - Loss: 1.6116
Epoch 10/50 - Loss: 1.6037
Epoch 11/50 - Loss: 1.5960
Epoch 12/50 - Loss: 1.5883
Epoch 13/50 - Loss: 1.5806
Epoch 14/50 - Loss: 1.5731
Epoch 15/50 - Loss: 1.5656
Epoch 16/50 - Loss: 1.5582
Epoch 17/50 - Loss: 1.5509
Epoch 18/50 - Loss: 1.5437
Epoch 19/50 - Loss: 1.5365
Epoch 20/50 - Loss: 1.5294
Epoch 21/50 - Loss: 1.5223
Epoch 22/50 - Loss: 1.5153
Epoch 23/50 - Loss: 1.5084
Epoch 24/50 - Loss: 1.5016
Epoch 25/50 - Loss: 1.4948
Epoch 26/50 - Loss: 1.4881
Epoch 27/50 - Loss: 1.4814
Epoch 28/50 - Loss: 1.4748
Epoch 29/50 - Loss: 1.4683
Epoch 30/50 - Loss: 1.4618
Epoch 31/50 - Loss: 1.4554
Epoch 32/50 - Loss: 1.4490
Epoch 33/50 - Loss: 1.4427
Epoch 34/50 - Loss: 1.4364
Epoch 35/50 - Loss: 1.4302
Epoch 36/50 - Loss: 1.4241
Epoch 37/50 - Loss: 1.4180
Epoch 38/5

In [54]:
# model evaluation
with torch.no_grad(): # no gradient tracking - no updates
    # foreward pass on test set
    y_test_pred = model.forward_pass(X_test_tensor)
    # convert probabilities to class labels
    y_test_pred_labels = (y_test_pred.squeeze() >= 0.5).int()

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test_tensor, y_test_pred_labels)
precision = precision_score(y_test_tensor, y_test_pred_labels)
recall = recall_score(y_test_tensor, y_test_pred_labels)
f1 = f1_score(y_test_tensor, y_test_pred_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7632
Precision: 0.8438
Recall: 0.7606
F1 Score: 0.8000
